# Hint Strategy 1
This notebook demonstrates how an LLM performs when presented with a Javaee -> Quarkus JMS issue and provide a hint that can be used to improve the basic rule.
Installing pre requisites and configuring Kai with `GPT-4o-mini`

In [1]:
%pip uninstall kai -y
%pip install --no-cache-dir git+https://github.com/konveyor/kai.git@main
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/konveyor/kai.git (to revision main) to /private/var/folders/4f/bv2hh9jd44b94_4zf2qby2sh0000gn/T/pip-req-build-w9h_3njt
  Running command git clone --filter=blob:none --quiet https://github.com/konveyor/kai.git /private/var/folders/4f/bv2hh9jd44b94_4zf2qby2sh0000gn/T/pip-req-build-w9h_3njt
  error: 1017 bytes of body are still expected
  fetch-pack: unexpected disconnect while reading sideband packet
  fatal: early EOF
  fatal: fetch-pack: invalid index-pack output
  fatal: could not fetch bc948d62498a2ca123c6c9426b5dc88876ebf7f5 from promisor remote
  You can inspect what was checked out with 'git status'
  and retry with 'git restore --source=HEAD :/'

  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/konveyor/kai.git /private/var/folders/4f/bv2hh9jd44b94_4zf2qby2sh0000gn/T/pip-req-build-w9h_3njt did not run successfully.
  │ exit code: 128


In [2]:
from IPython.display import display, Markdown
from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders
from dotenv import load_dotenv
load_dotenv() 

# Initialize the model provider using GPT-4o via OpenAI
model = ModelProvider.from_config(KaiConfigModels(
    provider=SupportedModelProviders.CHAT_OPENAI,
    args={"model": "gpt-3.5-turbo",
          "temperature": 0.1,},
))

# Async rendering function for displaying the response
async def rendered_llm_call(prompt: str):
    response = await model.ainvoke_llm(prompt)
    display(Markdown(response.content))
    return response



/Users/sraghuna/local_dev/kai-notebook-solution-server/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Below is the snippet we are trying to migrate from javaee to Qusrkus. This code uses JMS which needs to be migarted to Quarkus equivalent.
Our goal is to assess how well the LLM refactors this configuration without a hint, and whether it can replace deprecated patterns with the right approach.

In [3]:
before_code = """\
import javax.jms.* 

public class InventoryNotificationMDB implements MessageListener {

    private static final int LOW_THRESHOLD = 50;

    @Inject
    private CatalogService catalogService;

    private final static String JNDI_FACTORY = "weblogic.jndi.WLInitialContextFactory";
    private final static String JMS_FACTORY = "TCF";
    private final static String TOPIC = "topic/orders";
    private TopicConnection tcon;
    private TopicSession tsession;
    private TopicSubscriber tsubscriber;

    public void onMessage(Message rcvMessage) {
        TextMessage msg;
        {
            try {
                System.out.println("received message inventory");
                if (rcvMessage instanceof TextMessage) {
                    msg = (TextMessage) rcvMessage;
                    String orderStr = msg.getBody(String.class);
                    Order order = Transformers.jsonToOrder(orderStr);
                    order.getItemList().forEach(orderItem -> {
                        int old_quantity = catalogService.getCatalogItemById(orderItem.getProductId()).getInventory().getQuantity();
                        int new_quantity = old_quantity - orderItem.getQuantity();
                        if (new_quantity < LOW_THRESHOLD) {
                            System.out.println("Inventory for item " + orderItem.getProductId() + " is below threshold (" + LOW_THRESHOLD + "), contact supplier!");
                        } else {
                            orderItem.setQuantity(new_quantity);
                        }
                    });
                }


            } catch (JMSException jmse) {
                System.err.println("An exception occurred: " + jmse.getMessage());
            }
        }
    }
    }
"""

This rule flags suggests migrating javaee JMS referesnces with Quarkus specific ones.

In [4]:
hint_to_migrate="""\
    
  description:  References to JavaEE/JakartaEE JMS elements should be removed and replaced with their Quarkus SmallRye/Microprofile equivalents.
  message: |
   References to JavaEE/JakartaEE JMS elements should be removed and replaced with their Quarkus SmallRye/Microprofile equivalents.

    """

In [8]:
source="javaee"
target="quarkus, jakarta-ee"

In [9]:
prompt = f"""

The following java code needs to be migrated from {source} to {target}.

```java
{before_code}
```

Use the following hint's description and message to migrate. 

{hint_to_migrate}

Also, remove the deprecated and unused APIs and suggest the correct alternatives based on the migration needs. 

Can you provide a refactored version of {before_code} that is compatible with {target}?
Also, create a step by step process for the above migration? Summarize and create a hint with the changes.

"""

In [10]:

response = await rendered_llm_call(prompt)

To migrate the given code from JavaEE to Quarkus with Jakarta EE, we need to replace the JMS elements with their Quarkus SmallRye/Microprofile equivalents. Here is the refactored version of the code:

```java
import org.eclipse.microprofile.reactive.messaging.Incoming;

public class InventoryNotificationMDB {

    private static final int LOW_THRESHOLD = 50;

    @Inject
    private CatalogService catalogService;

    @Incoming("orders")
    public void onMessage(String orderStr) {
        Order order = Transformers.jsonToOrder(orderStr);
        order.getItemList().forEach(orderItem -> {
            int old_quantity = catalogService.getCatalogItemById(orderItem.getProductId()).getInventory().getQuantity();
            int new_quantity = old_quantity - orderItem.getQuantity();
            if (new_quantity < LOW_THRESHOLD) {
                System.out.println("Inventory for item " + orderItem.getProductId() + " is below threshold (" + LOW_THRESHOLD + "), contact supplier!");
            } else {
                orderItem.setQuantity(new_quantity);
            }
        });
    }
}
```

Step by step process for the migration:
1. Replace the import statement from `javax.jms.*` to `org.eclipse.microprofile.reactive.messaging.Incoming`.
2. Remove the JNDI_FACTORY, JMS_FACTORY, and TOPIC constants as they are not needed in Quarkus.
3. Remove the TopicConnection, TopicSession, and TopicSubscriber variables as they are not needed in Quarkus.
4. Change the `onMessage` method signature to accept a String parameter instead of a Message parameter.
5. Update the method logic to process the incoming message directly as a String.
6. Update the method logic to use the `@Incoming` annotation with the appropriate channel name ("orders" in this case).

Hint: References to JavaEE/JakartaEE JMS elements should be removed and replaced with their Quarkus SmallRye/Microprofile equivalents.